## AdaBoost

Adaboost是一种boosting算法，与boosting算法相似的有bagging，两者同属于ensemble learning。

把准确率不高但在50%以上的分类器成为弱分类器，弱分类器可以是一些基础的ML算法比如单层决策树等。对于统一数据可能有多种弱分类器，`AdaBoost`就是将这些弱分类器结合起来，成为基分类器，从而有效提高整体的准确率。

但是要想获得好的结合或者集成，对基分类器的要求是“好而不同”，即有一定的准确率，而且弱分类器之间要有“多样性”，比如一个判断是否为男性的任务，弱分类器1侧重从鼻子、耳朵这些特征判断是否是男人，分类器2侧重脸和眼睛等等，把这些分类器结合起来就有了所有用来判断是否男性的特征，并且adaboost还可以给每个基分类器赋值不同的权重，比如从脸比鼻子更能判断是否为男性，就可以把分类器2的权重调高一些，这也是adaboost需要学习的内容。

## 数学推导

`AdaBoost`可以表示为基分类器的线性组合： $$ H(\boldsymbol{x})=\sum_{i=1}^{N} \alpha_{i} h_{i}(\boldsymbol{x}) $$ 其中$h_i(x),i=1,2,...$表示基分类器，$\alpha_i$是每个基分类器对应的权重，表示如下： $$ \alpha_{i}=\frac{1}{2} \ln \left(\frac{1-\epsilon_{i}}{\epsilon_{i}}\right) $$ 其中$\epsilon_{i}$是每个弱分类器的错误率。


boosting是一族将弱学习器提升为强学习器的算法。这族算法的工作机制是：先从初试训练集训练出一个基学习器，再根据基学习器的表现对训练样本进行调整，使得先前基学习器做错的训练样本在后续受到更多关注，然后基于调整后的样本分布来训练下一个基分类器，如此重复进行，直至基学习器数目达到实现指定值T，最后将这T个基学习器进行加权组合。

boosting族算法最著名的代表室AdaBoost,AdaBoost算法有多种推导方式，比较容易理解的是基于“加性模型”，即基学习器的线性组合:

$$
H(x) = \sum_{t=1}^{T} \alpha_t h_t(x)
$$

指数损失函数为：

$$l_{\exp}(H \mid D) = \mathbb{E}_{X \sim D} \left[ e^{-f(x) H(x)} \right]$$

#### 算法步骤

**输入：**
训练集
$$D = (x_1, y_1), (x_2, y_2), \ldots, (x_m, y_m)$$
**基学习算法**
$$\xi$$
**训练轮次**
$$T$$


#### 过程

1. **初始化权重分布**


$$D_1(x) = \frac{1}{m}$$


2. **for**


$$t = 1,2,3,\ldots,T$$


**do:**

3. 训练弱分类器


$$h_t = \xi(D, D_t)$$


4. 计算分类误差


$$\epsilon_t = \mathbb{P}_{X \sim D_t}(h_t(x) \neq f(x))$$


5. 若


$$\epsilon_t > 0.5$$


则 **break**

6. 计算弱分类器系数


$$\alpha_t = \frac{1}{2} \ln\left(\frac{1 - \epsilon_t}{\epsilon_t}\right)$$


7. 更新分布


$$D_{t+1}(x) = \frac{ D_t(x) \exp(-\alpha_t f(x) h_t(x)) }{ Z_t }$$


8. **end for**


## 输出

最终分类器：

$$H(x) = \operatorname{sign}\left( \sum_{t=1}^{T} \alpha_t h_t(x) \right)$$

## 补充说明

其中：

* ( \epsilon_t ) 是弱分类器在分布 (D_t) 下的错误率
* ( Z_t ) 为归一化因子，用以确保 (D_{t+1}) 是合法概率分布





In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#构建并训练决策树分类器，这里特征选择标准使用基尼指数，树的最大深度为1
base_model = DecisionTreeClassifier(max_depth = 1, criterion='gini', random_state=1).fix(X_train, y_train)
y_pred = base_model.predict(X_test)
print(f"决策树的准确率：{accuracy_score(y_test, y_pred):.3f}")

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd

wine = load_wine() #使用葡萄酒数据集
print(f"所有特征：{wine.feature_names}")
X = pd.DataFrame(wine.data, columns = wine.feature_names)
y = pd.Series(wine.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

print(f"训练数据量：{len(X_train)}, 测试数据量：{len(X_test)}")

# 定义模型，这里最大分类器数量为50，学习率为1.5
model = AdaBoostClassifier(base_estimator = base_model, n_estimators = 50, learning_rate = 0.8)
# 训练
model.fit(X_train, y_train)
# 预测
y_pred = model.predict(X_test)
acc = metrics.accuracy_score(y_test, y_pred)
print(f"准确率：{acc:.2}")

### 使用GridSearchCV自动调参

In [ ]:
hyperparameter_space = {'n_estimators': list(range(2, 102, 2)),
                        'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

# 使用准确率为标准，将得到的准确率最高的参数输出，cv=5表示交叉验证参数，这里使用五折交叉验证，n_jobs=-1表示并行数和cpu一致
gs = GridSearchCV(AdaBoostClassifier(
    algorithm = 'SAMME.R',
    random_state=1),
    param_grid=hyperparameter_space,
    scoring = "accuracy", n_jobs=-1, cv = 5)

gs.fit(X_train, y_train)
print("最优超参数：", gs.best_params_)